In [ ]:
!pip install -U -q PyDrive 
  
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
  
  
# Authenticate and create the PyDrive client. 
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)

In [ ]:

from os import path
url = "https://quera.org/contest/assignments/39253/download_problem_initial_project/132527/?noconvert=true"

!wget https://quera.org/contest/assignments/39253/download_problem_initial_project/132527/?noconvert=true

--2022-03-03 05:40:30--  https://quera.org/contest/assignments/39253/download_problem_initial_project/132527/?noconvert=true
Resolving quera.org (quera.org)... 185.143.234.61, 185.143.233.61
Connecting to quera.org (quera.org)|185.143.234.61|:443... ^C


In [1]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/test.csv           
  inflating: data/train.csv          


In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

ModuleNotFoundError: No module named 'torchvision'

In [2]:
import pandas as pd

df = pd.read_csv("data/train.csv")



price_y = df["price"]

#df.drop(columns = ["price"], inplace = True)
print(len(df))
df.head()

68840


,id,product_description,price
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر\r\nپورت ی...,1080000
1,1,{'مشخصات فنی': 'CABLE LENTH:\r\n10feet \\/3M\r...,990000
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB\r\nطول کابل:...,1680000
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000


In [3]:
test_df = pd.read_csv("data/test.csv")

test_df.head()

,id,product_description
0,0,"{'سازگار با': 'C21N1401\r\nX455', 'ولتاژ باتری..."
1,1,"{'سازگار با': 'A31N1519\r\nX540', 'ولتاژ باتری..."
2,2,"{'تعداد فن': 'ندارد', 'نوع سیستم خنک کنندگی': ..."
3,3,{'سازگار با': 'A41N1424\r\nGL552\r\nZX50\r\nAS...
4,4,"{'سازگار با': 'AP31\r\nASUS 1008', 'ولتاژ باتر..."


In [4]:
train_test_df = pd.concat([df, test_df], ignore_index=True)

In [5]:
import ast
train_test_df['product_description'] = train_test_df['product_description'].apply(lambda x: ast.literal_eval(x))

In [6]:
train_test_df['keys'] = train_test_df['product_description'].apply(lambda x: x.keys())
train_test_df.loc[0][2]

1080000.0

In [7]:
train_test_df.loc[5][1]['مشخصات فنی']

'مدل: CAB-CONSOLE-RJ45\r\nتوضيحات: Cisco RJ45 Male to DB9 Female 6ft Console Cable 72-3383-01\r\nكانكتور 1: RJ-45 – male\r\nكانكتور 2: 9-pin D-Sub (DB-9) – female\r\nطول: 180سانتی متر\r\nنوع كابل: Serial cable '

In [8]:
train_test_df["brand"] = train_test_df['product_description'].apply(lambda x: x['برند'])
train_test_df["category"] = train_test_df['product_description'].apply(lambda x: x['دسته بندی'])
train_test_df.head(10)

,id,product_description,price,keys,brand,category
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر پورت یو ...,1080000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو
1,1,{'مشخصات فنی': 'CABLE LENTH: 10feet \/3M PC ...,990000.0,"(مشخصات فنی, دسته بندی, برند)",ادیمکس,کابل کمبو
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB طول کابل: 1...,1680000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000.0,"(دسته بندی, برند)",متفرقه,کابل کمبو
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000.0,"(مشخصات فنی, دسته بندی, برند)",دی-لینک,کابل کمبو
5,5,{'مشخصات فنی': 'مدل: CAB-CONSOLE-RJ45 توضيحات...,470000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو
6,6,"{'کارکرد': 'در دستگاه های ریسوگراف', 'دستگاه‌ه...",4950000.0,"(کارکرد, دستگاه‌های سازگار, سایر توضیحات, حجم,...",متفرقه,ریسوگراف
7,7,"{'کارکرد': '12000-15000 برگ', 'دستگاه‌های سازگ...",1840000.0,"(کارکرد, دستگاه‌های سازگار, حجم, دسته بندی, برند)",متفرقه,ریسوگراف
8,8,"{'کارکرد': 'در دستگاه های ریکو', 'دستگاه‌های س...",840000.0,"(کارکرد, دستگاه‌های سازگار, سایر توضیحات, حجم,...",متفرقه,ریسوگراف
9,9,"{'کارکرد': 'در دستگاه های ریسوگراف', 'دستگاه‌ه...",6950000.0,"(کارکرد, دستگاه‌های سازگار, سایر توضیحات, حجم,...",متفرقه,ریسوگراف


In [9]:
def len_feature(x):

  if('مشخصات فنی' in x):
    return x['مشخصات فنی'].count(":")
  else:
    return 0

train_test_df["num_features"] = train_test_df['product_description'].apply(lambda x: len_feature(x))
train_test_df.head()

,id,product_description,price,keys,brand,category,num_features
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر پورت یو ...,1080000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو,1
1,1,{'مشخصات فنی': 'CABLE LENTH: 10feet \/3M PC ...,990000.0,"(مشخصات فنی, دسته بندی, برند)",ادیمکس,کابل کمبو,3
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB طول کابل: 1...,1680000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو,2
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000.0,"(دسته بندی, برند)",متفرقه,کابل کمبو,0
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000.0,"(مشخصات فنی, دسته بندی, برند)",دی-لینک,کابل کمبو,4


In [10]:
train_test_df.loc[1]['product_description']

{'مشخصات فنی': 'CABLE LENTH:\r\n10feet \\/3M\r\nPC SIDE CONNECTORS_CONNECTED TO PC : \r\nPS\\/2 Keyboard mini-Din 6 pin, male\r\nPS\\/2 Mouse Mini-Din 6 pin, male\r\nVGA HDDB 15pin, male\r\nKVM SIDE CONNECTORS-CONNECTED TO KVM SWITCH: \r\nHDDB 15pin, male',
 'دسته بندی': 'کابل کمبو',
 'برند': 'ادیمکس'}

In [11]:
import re
from __future__ import unicode_literals

re.findall(u"-?\d+\.?\d*feet", train_test_df.loc[1]['product_description']["مشخصات فنی"])

['10feet']

In [12]:
import re
not_found_list = []
def cable_length(x):

  if("مشخصات فنی" in x):
    x = x["مشخصات فنی"]
  else:
    return 0
  if(re.findall(u"-?\d+\.?\d* سانتی متر", x)):
    y = re.findall(u"-?\d+\.?\d* سانتی متر", x)[0]
    return float(re.findall(u"-?\d+\.?\d*", y)[0])
  elif(re.findall(u"-?\d+\.?\d*سانتی متر", x)):
    y = re.findall(u"-?\d+\.?\d*سانتی متر", x)[0]
    return float(re.findall(u"-?\d+\.?\d*", y)[0])
  elif(re.findall(u"-?\d+\.?\d* متر", x)):
    y = re.findall(u"-?\d+\.?\d* متر", x)[0]
    return float(re.findall(u"-?\d+\.?\d*", y)[0])*100
  elif(re.findall(u"-?\d+\.?\d*متر", x)):
    y = re.findall(u"-?\d+\.?\d*متر", x)[0]
    return float(re.findall(u"-?\d+\.?\d*", y)[0])*100
  elif(re.findall(u"-?\d+\.?\d* feet", x)):
    y = re.findall(u"-?\d+\.?\d* feet", x)[0]
    return float(re.findall(u"-?\d+\.?\d*", y)[0])*30.48
  elif(re.findall(u"-?\d+\.?\d*feet", x)):
    y = re.findall(u"-?\d+\.?\d*feet", x)[0]
    return float(re.findall(u"-?\d+\.?\d*", y)[0])*30.48
  else:
    not_found_list.append(x)
    return 0

  







train_test_df["cable_length"] = train_test_df['product_description'].apply(lambda x: cable_length(x))

In [13]:
len(not_found_list)

2485

In [14]:
train_test_df.head()

,id,product_description,price,keys,brand,category,num_features,cable_length
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر پورت یو ...,1080000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو,1,150.0
1,1,{'مشخصات فنی': 'CABLE LENTH: 10feet \/3M PC ...,990000.0,"(مشخصات فنی, دسته بندی, برند)",ادیمکس,کابل کمبو,3,304.8
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB طول کابل: 1...,1680000.0,"(مشخصات فنی, دسته بندی, برند)",متفرقه,کابل کمبو,2,150.0
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000.0,"(دسته بندی, برند)",متفرقه,کابل کمبو,0,0.0
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000.0,"(مشخصات فنی, دسته بندی, برند)",دی-لینک,کابل کمبو,4,300.0


In [15]:
list_brand = train_test_df['brand'].value_counts().index.tolist()
brand_dict = { x:i for i,x in enumerate(list_brand)}
train_test_df["brand"] = train_test_df["brand"].apply(lambda x: brand_dict[x] if x in brand_dict.keys() else n)
train_test_df.head()

,id,product_description,price,keys,brand,category,num_features,cable_length
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر پورت یو ...,1080000.0,"(مشخصات فنی, دسته بندی, برند)",0,کابل کمبو,1,150.0
1,1,{'مشخصات فنی': 'CABLE LENTH: 10feet \/3M PC ...,990000.0,"(مشخصات فنی, دسته بندی, برند)",607,کابل کمبو,3,304.8
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB طول کابل: 1...,1680000.0,"(مشخصات فنی, دسته بندی, برند)",0,کابل کمبو,2,150.0
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000.0,"(دسته بندی, برند)",0,کابل کمبو,0,0.0
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000.0,"(مشخصات فنی, دسته بندی, برند)",48,کابل کمبو,4,300.0


In [16]:
list_category = train_test_df['category'].value_counts().index.tolist()
category_dict = { x:i for i,x in enumerate(list_category)}
train_test_df["category"] = train_test_df["category"].apply(lambda x: category_dict[x] if x in category_dict.keys() else n)

In [14]:
n = 30
list_brand = train_test_df['brand'].value_counts()[:n].index.tolist()
brand_dict = { x:i for i,x in enumerate(list_brand)}
train_test_df["brand"] = train_test_df["brand"].apply(lambda x: brand_dict[x] if x in brand_dict.keys() else n)
train_test_df.head()

,id,product_description,price,keys,brand,category,num_features,cable_length
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر پورت یو ...,1080000.0,"(مشخصات فنی, دسته بندی, برند)",0,کابل کمبو,1,150.0
1,1,{'مشخصات فنی': 'CABLE LENTH: 10feet \/3M PC ...,990000.0,"(مشخصات فنی, دسته بندی, برند)",30,کابل کمبو,3,304.8
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB طول کابل: 1...,1680000.0,"(مشخصات فنی, دسته بندی, برند)",0,کابل کمبو,2,150.0
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000.0,"(دسته بندی, برند)",0,کابل کمبو,0,0.0
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000.0,"(مشخصات فنی, دسته بندی, برند)",30,کابل کمبو,4,300.0


In [15]:
n = 30
list_category = train_test_df['category'].value_counts()[:n].index.tolist()
category_dict = { x:i for i,x in enumerate(list_category)}
train_test_df["category"] = train_test_df["category"].apply(lambda x: category_dict[x] if x in category_dict.keys() else n)

In [13]:
y = train_test_df.drop(columns = ["id", "price", "product_description", 'keys'])
y

,brand,category,num_features,cable_length
0,0,179,1,150.0
1,607,179,3,304.8
2,0,179,2,150.0
3,0,179,0,0.0
4,48,179,4,300.0
...,...,...,...,...
103097,2,0,0,0.0
103098,0,0,0,0.0
103099,7,0,0,0.0
103100,0,0,0,0.0


In [17]:
y = pd.get_dummies(train_test_df, columns=['brand', 'category'], prefix=['Brand', 'category']).drop(columns = ["id", "price", "product_description", 'keys'])
y

,num_features,cable_length,Brand_0,Brand_1,Brand_2,Brand_3,Brand_4,Brand_5,Brand_6,Brand_7,...,category_170,category_171,category_172,category_173,category_174,category_175,category_176,category_177,category_178,category_179
0,1,150.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,304.8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,150.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,300.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103097,0,0.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103098,0,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103099,0,0.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
103100,0,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
from scipy import sparse

y = sparse.csr_matrix(y.values)
x_train = y[0:68840]
x_test = y[68840:]

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
GBR = GradientBoostingRegressor()
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }
                 

In [ ]:
pipe = Pipeline(steps=[('Scalar', StandardScaler()), ('SVM', SVC())])

param_grid = {'SVM__C': [0.1],
              'SVM__gamma': [1],
              'SVM__kernel': ['rbf']}
 

In [14]:
x_train = y.loc[0:68839].values
x_test = y.loc[68840:].values

In [ ]:
svc_model = SVC(C=1, kernel="rbf", gamma="scale")
svc_model.fit(x_train, price_y)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(x_train)
print(classification_report(price_y, predictions))

In [ ]:
grid = GridSearchCV(pipe, param_grid, cv = 4, verbose = 3, n_jobs = -1)
grid.fit(x_train, price_y.values)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid.best_estimator_)
print("\n The best score across ALL searched params:\n",grid.best_score_)
print("\n The best parameters across ALL searched params:\n",grid.best_params_)

In [ ]:
grid_GBR = GridSearchCV(estimator=GBR, param_grid = parameters, cv = 2, n_jobs=-1, verbose = 3)
grid_GBR.fit(x_train, price_y)

In [19]:
from sklearn.model_selection import train_test_split
import sklearn
import seaborn as sns

In [22]:
train_X, test_X, train_y, test_y = train_test_split(x_train, price_y, test_size=0.10, random_state=0)


In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
 

X_train = sc.fit_transform(train_X)
X_test = sc.transform(test_X)

In [ ]:
from sklearn.decomposition import PCA
 
pca = PCA(n_components = 20)
 
X_train = pca.fit_transform(train_X)
X_test = pca.transform(test_X)
 
explained_variance = pca.explained_variance_ratio_

In [ ]:
explained_variance

array([0.0559374 , 0.05544951, 0.05511998, 0.05287868, 0.05104168,
       0.05087531, 0.05054775, 0.05042241, 0.05019751, 0.0499501 ,
       0.04920638, 0.04893961, 0.0485314 , 0.04816392, 0.04795232,
       0.04741729, 0.04724052, 0.04705785, 0.04664254, 0.04642784])

In [40]:
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
model.fit(train_X, train_y) 

RandomForestRegressor(random_state=0)

In [28]:
from sklearn.svm import SVR
model = SVR(kernel="rbf", C=100, gamma=0.1, epsilon=0.1).fit(X_train, train_y) 

In [ ]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier

# Definition of the Radial Basis Function (RBF) as kernel
kernel = RBF( length_scale=1.0 , length_scale_bounds=(0.1, 10.0))

# Train Gaussian Process
model = GaussianProcessClassifier(kernel=kernel, random_state=0).fit(train_X, train_y)

In [19]:
from sklearn.tree import DecisionTreeRegressor

# Build decision tree
model=DecisionTreeRegressor(max_depth=500)
model.fit(X_train,train_y)

DecisionTreeRegressor(max_depth=500)

In [ ]:
from sklearn.linear_model import LinearRegression 

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

# define the pipeline and train model
model = Pipeline([('poly', PolynomialFeatures(degree=3)),
                  ('linear', LinearRegression(fit_intercept=False))])
                  
model.fit(X_train, train_y.values)

# print out regression coefficients
print(model.named_steps['linear'].coef_)    
print(model.named_steps['linear'].intercept_)  

In [31]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)
model.fit(X_train,train_y)

KNeighborsRegressor()

In [32]:
y_pred = model.predict(X_train)

In [33]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
print("MAE",mean_absolute_error(train_y,y_pred))
print("MAPE",mean_absolute_percentage_error(train_y, y_pred))


MAE 8815582.240583641
MAPE 1.2823367300802732e+20


In [30]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
print("MAE",mean_absolute_error(train_y,y_pred))
print("MAPE",mean_absolute_percentage_error(train_y, y_pred))


MAE 15062835.698862353
MAPE 1.66046490334853e+19


In [ ]:
test_y.values

array([  550000,   950000,  5900000, ...,   475000, 91200000,  7995000])

In [ ]:
import numpy as np
def calculate_mape(actual, predicted):
  
    # Convert actual and predicted
    # to numpy array data type if not already
    if not all([isinstance(actual, np.ndarray),
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), 
        np.array(predicted)
  
    # Calculate the MAPE value and return
    return round(np.mean(np.abs((
      actual - predicted) / actual)) * 100, 2)
print("MAPE",calculate_mape(test_y.values,y_pred))


MAPE inf


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]


In [ ]:
model1.predict(train_X)

In [ ]:
out_x = pca.transform(sc.transform(y.loc[68840:].values))

In [24]:
out_x = pca.transform(y.loc[68840:].values)

NameError: ignored

In [28]:
out_y = model1.predict(sc.transform(y.loc[68840:].values))

In [29]:
out_y

array([[10662854. ],
       [10662854. ],
       [31890016. ],
       ...,
       [ 3615313. ],
       [ 3616690.5],
       [ 3635155.2]], dtype=float32)

In [55]:
out_y = [x[0] for x in out_y]

In [52]:
data = {"id": test_df["id"], "price":out_y}
df_out = pd.DataFrame(data)

df_out.to_csv("output.csv", index= False)

In [ ]:
tensor_X = torch.from_numpy(train_X).float() 
tensor_y = torch.from_numpy(train_y.values).float() 
print(tensor_X.shape, tensor_y.shape)

torch.Size([61956, 2018]) torch.Size([61956])


In [ ]:
tensor_y = tensor_y.unsqueeze(1)
print(tensor_X.shape, tensor_y.shape)

torch.Size([61956, 2018]) torch.Size([61956, 1])


In [ ]:
torch_dataset = Data.TensorDataset(tensor_X, tensor_y)


In [ ]:
batch = 16

In [ ]:
loader = Data.DataLoader(
    dataset = torch_dataset,      # torch TensorDataset format
    batch_size = batch,           # mini batch size
    shuffle=True,                 # random shuffle for training
    num_workers=2,                # subprocesses for loading data
)

In [ ]:
model = nn.Sequential(nn.Linear(2018, 1024), # first layer
   nn.ReLU(),
   nn.Linear(1024, 256), # second layer
   nn.ReLU(),
   nn.Linear(256, 32), # third layer
   nn.ReLU(),
   nn.Linear(32, 1)   # fourth layer
   )
model = model.cuda() 

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.000000001)
loss_function = torch.nn.MSELoss()

In [ ]:
loss_lst = []
for epoch in range(1, 5):
    batch = 1
    print("Epoch", epoch)
    for step, (batch_x, batch_y) in enumerate(loader): 
        var_X, var_y = Variable(batch_x.cuda()), Variable(batch_y.cuda())
        prediction = model(var_X)
        rmse_loss = loss_function(prediction, var_y)

        optimizer.zero_grad()
        rmse_loss.backward()
        optimizer.step()
        
        loss_lst.append(float(rmse_loss))
        print("Batch: ", batch, ", loss: ", rmse_loss)
        batch += 1
    print("Result of the last epoch's batch: ", rmse_loss)
    print(' ')

In [ ]:
import time
from timeit import Timer as timer

In [23]:
def run_model1(X_train, y_train, X_valid, y_valid):
    '''- returns an MLP model trained on tfidf vectorized sparse input.
    - Does not perform best on binarized input.
    - Uses Adam optimizer with constant learning rate. 
    - trains 2 epochs, Batch size is doubled at every epoch to speed up the optimization'''

    model_in = ks.Input(shape=(X_train.shape[1],), dtype='float32', sparse=True)
    out = ks.layers.Dense(256, activation='relu')(model_in)
    # out = ks.layers.Dropout(0.1)(out)     ## performance is better without dropouts
    out = ks.layers.Dense(128, activation='relu')(out)
    # out = ks.layers.Dropout(0.1)(out)
    out = ks.layers.Dense(64, activation='relu')(out)
    # out = ks.layers.Dropout(0.2)(out)
    out = ks.layers.Dense(32, activation='relu')(out)
    out = ks.layers.Dense(1)(out)
    model = ks.Model(model_in, out)
    
    #mean_absolute_percentage_error
    model.compile(loss='mean_squared_error', optimizer=ks.optimizers.Adam(learning_rate=3e-3))
    
    
    return model

In [22]:
from tensorflow import keras as ks
import pandas as pd
import numpy as np
import scipy
import tensorflow as tf
from keras.models import load_model

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf


from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import KFold

In [46]:
ks.backend.set_epsilon(1)

In [24]:
model1 = run_model1(X_train, np.array(train_y, dtype='float32'), test_X, np.array(test_y, dtype='float32'))

In [25]:
model1.fit(x=X_train, y=np.array(train_y, dtype='float32'), batch_size=64, epochs=50, verbose=1, validation_data=(X_test, np.array(test_y, dtype='float32')))


Epoch 1/50
969/969 [==============================] - 6s 4ms/step - loss: 4102577994072064.0000 - val_loss: 5329401042436096.0000
Epoch 2/50
969/969 [==============================] - 4s 4ms/step - loss: 4022512019046400.0000 - val_loss: 5329298500091904.0000
Epoch 3/50
969/969 [==============================] - 4s 4ms/step - loss: 4020857919766528.0000 - val_loss: 5308732586065920.0000
Epoch 4/50
969/969 [==============================] - 4s 4ms/step - loss: 4018940485304320.0000 - val_loss: 5316325551374336.0000
Epoch 5/50
969/969 [==============================] - 4s 4ms/step - loss: 4017448252604416.0000 - val_loss: 5326540594216960.0000
Epoch 6/50
969/969 [==============================] - 4s 4ms/step - loss: 4017120224477184.0000 - val_loss: 5334930275958784.0000
Epoch 7/50
969/969 [==============================] - 4s 4ms/step - loss: 4013151674695680.0000 - val_loss: 5323536264593408.0000
Epoch 8/50
969/969 [==============================] - 4s 4ms/step - loss: 4008381845078016

In [ ]:
model1.predict(train_X)